## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Содержание проекта

- [Знакомство с данными и их подготовка](#the_destination)
- [Предобработка данных](#the_destination_1)
- [Лемматизация](#the_destination_6)
- [Категоризация данных](#the_destination_4)
- [Промежуточные выводы](#the_destination_5)
- [Общий вывод](#the_destination_3)

<a id='the_destination'></a> 

## Знакомство с данными и их подготовка

In [1]:
import pandas as pd
debtor_data = pd.read_csv('/datasets/data.csv')

In [2]:
print(debtor_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [3]:
display(debtor_data.describe())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
display(debtor_data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
#for element in debtor_data:
 #   display (debtor_data[element].value_counts())

### Вывод

в столбцах days_employed и total_income есть нулевые значения.
 - Отрицательные значения в столбцах children и days_employed,чего не может быть.
 - В некоторых столбцах отсутствие данных обозначено через 0 или XNA, используются прописные буквы. 
 - Пропуски могут быть как случайными, так и нет. В столбце days_employed пропуски могут быть связаны с отсутствием данных от заемщика. 
 - В столбце total_income хранятся важные для банка данные, которые заемщики обычно подтверждают, поэтому отсутствие данных может быть связано со сбоями или ошибками.
 - В столбце education много дубликатов.


<a id='the_destination_1'></a> 

## Предобработка данных

#### Обработка пропусков

In [6]:
print(debtor_data['days_employed'].isnull().sum())
print(debtor_data['total_income'].isnull().sum())

2174
2174


Нулевые значения есть в столбце days_employed, но данные значения не будут использованы при анализе данных, поэтому изменять их не будем
Нулевые значения в total_income заменим медианным значением

In [7]:
#выведем уникальные значения занятости
inc_type_unique = debtor_data['income_type'].unique()
print(inc_type_unique)

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [8]:
display(debtor_data[debtor_data['income_type'] == 'безработный'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [9]:
#найдем медианные значения в каждой группе занятости
inc_type_grouped_median = debtor_data.groupby('income_type')['total_income'].median()
print(inc_type_grouped_median)

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [10]:
#заполним пустые значения в каждой группе занятости
for inc_type in inc_type_unique:
    inc_mean = inc_type_grouped_median.loc[inc_type]
    debtor_data.loc[debtor_data['income_type'] == inc_type, 'total_income'] = debtor_data.loc[debtor_data['income_type'] == inc_type, 'total_income'].fillna(inc_mean)
print(debtor_data['total_income'].isnull().sum())

0


### Вывод

Заменили в столбце total_income нулевые значения на медианные для каждой группы занятости. Проверили, что в данном столбце нулевых значений не осталось. Нулевые значения могут быть потому, что данные были не предоставлены или не подтверждены документами.
Нулевые значения в days_employed могут быть также связаны с непредоставлением данных (проверили, что у 'безработный' есть данные в'days_employed').
Пропуски также могут появляться в результате сбоев при выгрузке данных.

#### Замена типа данных

In [11]:
#чтобы заменить тип данных на int заменили пропущенные значению на 0. Считаю это не критичным, тк этот столбец 
#не планирую использовать в расчетах
debtor_data['days_employed'] = debtor_data['days_employed'].fillna(0)
debtor_data['days_employed'] = debtor_data['days_employed'].astype('int')

In [12]:
debtor_data['total_income'] = debtor_data['total_income'].astype('int')

In [13]:
print(debtor_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


###### Вывод

Заминили вещественный тип данных на целочисленный в столбце days_employed, предварительно заменив пропущенные значения нулями.
Также заменили вещественный тип на целочисленный в столбце total_income.


#### Обработка дубликатов

In [14]:
#приведем все записи в столбце 'education' в нижний регистр
debtor_data['education'] = debtor_data['education'].str.lower()
print(debtor_data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [15]:
#проверим данные на наличие дублей, удалим их и сделаем повторную проверку на дубли
print(debtor_data.duplicated().sum())
debtor_data_clean = debtor_data.drop_duplicates().reset_index(drop=True)
print(debtor_data_clean.duplicated().sum())

71
0


In [16]:
print(debtor_data_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


###### Вывод

Методом drop_duplicates удалили полные дубликаты из таблицы. Такие дубликаты могли появиться при копировании данных или в результате ошибки при их добавлении в таблицу. 
Часть дубликатов появилась из-за по-разному заполненных значений в столбце 'education', но имеющих одинаковый смысл. Такие дубликаты были обработаны приведением всех записей к написанию без заглавных букв. 

<a id='the_destination_6'></a> 

## Лемматизация

In [17]:
from pymystem3 import Mystem
m = Mystem()
def lemmas_list(row):
    purpose_list = row['purpose']
    lemmas = m.lemmatize(purpose_list)
    return lemmas

debtor_data_clean['purpose_lemm']=debtor_data_clean.apply(lemmas_list, axis=1)
print(debtor_data_clean['purpose_lemm'].value_counts())

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

###### Вывод

Цели у заемщиков сформулированы по разному. Для дальнейшей категоризации разбили столбец целей на леммы. 

<a id='the_destination_4'></a> 

## Категоризация данных

In [18]:
def purpose_type(row):
    purpose_type_word = row['purpose_lemm'] 
    if 'автомобиль' in purpose_type_word:
        return 'Покупка aвтомобиля'
    if 'образование' in purpose_type_word:
        return 'Образование'
    if 'свадьба' in purpose_type_word:
        return 'Свадьба'
    
    return 'Покупка жилья'

debtor_data_clean['purpose_words'] = debtor_data_clean.apply(purpose_type, axis=1)
print(debtor_data_clean['purpose_words'].value_counts())

Покупка жилья         10811
Покупка aвтомобиля     4306
Образование            4013
Свадьба                2324
Name: purpose_words, dtype: int64


###### Вывод

Выделили 4 цели заема средств: покупка жилья, покупка автомобиля, образование, свадьба. Наибольшее число заемов на покупку жилья. Кредит на покупку автомобиля и образование берут почти в 2,5 раза реже, а на организацию свадьбы в 4,7 раз реже.

In [19]:
# определим величину квантилей 0.5, 0.25 и 0.75
quantile025 = debtor_data_clean['total_income'].quantile(0.25)
quantile05 = debtor_data_clean['total_income'].quantile(0.5)
quantile075 = debtor_data_clean['total_income'].quantile(0.75)
print('Квантиль 0.25:', quantile025)
print('Квантиль 0.50:', quantile05)
print('Квантиль 0.75:', quantile075)


Квантиль 0.25: 107623.0
Квантиль 0.50: 142594.0
Квантиль 0.75: 195820.25


In [20]:
#напишем функцию для введения категорий по уровню дохода
def income_category(income):
    if income < quantile025:
        return 'low25'
    if income < quantile05 and income > quantile025:
        return 'low50'
    if income < quantile075 and income > quantile05:
        return 'low75'
    return 'more75'

debtor_data_clean['income_category'] = debtor_data_clean['total_income'].apply(income_category)
print(debtor_data_clean['income_category'].value_counts())

more75    6434
low25     5364
low75     5247
low50     4409
Name: income_category, dtype: int64


###### Вывод

Определили квантили 0.25, 0.5 и 0.75 по значениям столбца total_income. Опираясь на эти значения разбили доход на 4 группы. Проверили методом value_counts(), что размер групп однороден и репрезентативен.

Исправление неверных данных
----
в данном разделе исправим неправильные значения в столбце children: 
  - в строке с количеством детей  "-1" исправим на 1
  - строки с количеством детей "20" изменим на 2, таких значений всего 76 и  такая замена не окажет серьезного влияния на результаты анализа 

In [21]:
debtor_data_clean['children'].replace(-1, 1, inplace=True)
debtor_data_clean['children'].replace(20, 2, inplace=True)
print(debtor_data_clean['children'].value_counts())

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
debt_quant_children = debtor_data_clean.groupby('children')['debt'].mean()*100
print(debt_quant_children)

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


In [23]:
table1 = pd.pivot_table(debtor_data_clean, index=['children'], values = 'debt',  aggfunc='mean')
format_dict = { 'debt': '{:.2%}'}
table1.style.format(format_dict)

,debt
children,
0,7.54%
1,9.17%
2,9.49%
3,8.18%
4,9.76%
5,0.00%


<a id='the_destination_5'></a> 

## Промежуточные выводы

Зависимость между наличием детей и возвратом кредита в срок есть.
 
- Процент Заемщиков без детей, возвращающих кредит не в срок составляет 7.5%. 
  Всех заемщиков с детьми объеденим в одну группу и расчитаем средний процент заемщиков, задерживающих выплаты по кредитам - 9.1%. Заемщиков с 5 детьми не учитывали, так как выборка таких клиентов не репрезентативная.
  Таким образом, наличие детей приводит к увеличению процента заемщиков, задерживающих выплаты на 1.6%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
debt_quantity_family = debtor_data_clean.groupby('family_status')['debt'].mean()*100
print(debt_quantity_family)

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
Name: debt, dtype: float64


In [25]:
#проверим как изменятся показатели в группах семейного положения в случае наличия детей
debt_quantity_family_children = debtor_data_clean[debtor_data_clean['children'] != 0].groupby('family_status')['debt'].mean()*100
print(debt_quantity_family_children)

family_status
Не женат / не замужем    11.678832
в разводе                 7.299270
вдовец / вдова            8.928571
гражданский брак         11.189303
женат / замужем           8.519811
Name: debt, dtype: float64


###### Вывод

Заемщики с семейным положением "женат / замужем" или ранее бывавшие в оформленных отношениях ("вдовец / вдова", "в разводе") в среднем возвращают кредит в срок на 2,5% чаще, чем заемщики, не состоящие в официальных отношения ("Не женат / не замужем", "гражданский брак"). 
Средние проценты существования задолжности у заемщиков находящихся, или бывших в официальных отношениях и у заемщиков без официально оформленных отношений составляют 6.9% и 9.5%, соответственно. 
В случае наличия у заемщиков с семейным положением "Не женат / не замужем" и "гражданский брак" детей, задержки по кредитам в группе увеличиваются в среднем с 9,5% до 11,4%. 
Наличие детей приводит к увеличению доли задержек по выплатам кредита минимум на 1% во всех группах, кроме "в разводе".

In [26]:
debt_quantity_income = debtor_data_clean.groupby('income_category')['debt'].mean()*100
print(debt_quantity_income)

income_category
low25     7.960477
low50     8.732139
low75     8.538212
more75    7.475909
Name: debt, dtype: float64


In [27]:
table2 = pd.pivot_table(debtor_data_clean, index=['income_category'], values = 'debt',  aggfunc='mean')
format_dict = { 'debt': '{:.2%}'}
table2.style.format(format_dict)

,debt
income_category,
low25,7.96%
low50,8.73%
low75,8.54%
more75,7.48%


In [28]:
print('Квантиль 0.25:', quantile025)
print('Квантиль 0.50:', quantile05)
print('Квантиль 0.75:', quantile075)

Квантиль 0.25: 107623.0
Квантиль 0.50: 142594.0
Квантиль 0.75: 195820.25


###### Вывод

Между возвратом кредита в срок и уровнем дохода есть связь.

- Заемщики с уровнем дохода меньше 107623.0 (квантиль 0,25 выборки) и больше 195820.25 (квантиль 0,75 выборки) на 1% реже возвращают кредит не в срок (7.96% и 7.48% должников, соответственно).
- В остальных группах доля должников составляет в среднем 8.6%.

- Как разные цели кредита влияют на его возврат в срок?

In [29]:
debt_quantity_purpose = debtor_data_clean.groupby('purpose_words')['debt'].mean()*100
print(debt_quantity_purpose)

purpose_words
Образование           9.220035
Покупка aвтомобиля    9.359034
Покупка жилья         7.233373
Свадьба               8.003442
Name: debt, dtype: float64


In [30]:
table2 = pd.pivot_table(debtor_data_clean, index=['purpose_words'], values = 'debt',  aggfunc='mean')
format_dict = { 'debt': '{:.2%}'}
table2.style.format(format_dict)

,debt
purpose_words,
Образование,9.22%
Покупка aвтомобиля,9.36%
Покупка жилья,7.23%
Свадьба,8.00%


###### Вывод

Заемщики с целью покупки жилья имеют наименьшую долю задержек по кредитам (7.2%). Доля заемщиков с целью 'образование' и 'покупка автомобиля' с возвратом кредита не в срок больше примерно на 2% и составляет 9.2% и 9.4%, соответственно. Доля заемщиков с целью "свадьба", не возвративших кредит в срок - 8%.

<a id='the_destination_3'></a> 

## Общий вывод

Целью проекта было разобраться влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

- Для работы с данными была проведена их подготовка: заменены некорректные значения, удалены нулевые значения и дубликаты. 


Заемщики, состоящие в официальных отношениях или ранее в них бывавшие реже гасят кредит позже срока. 
* Доля заемщиков, гасивших кредит не в срок в этой группе составляет 6.9%. Доля заемщиков, гасивших кредит позже срока с семейным положением "не женат/ не замужем" и "в гражданском браке" больше на 2.6% и составляет 9.5%.
* Доля заемщиков гасивших кредит позже срока возрастает в случае наличия у него детей минимум на 0.7%. Для заемщиков без официально оформленных отношений наличие детей приводит к росту доли заемщиков, гасивших кредит не в срок на 2%. В группе же с официально оформленными отношениями или ранее в них бывавших и имеющих детей, процент заемщиков гасивших кредит не в срок увеличился на 1,3%.